In [258]:
import pandas as pd
import numpy as np

In [259]:
data1= pd.read_csv('/content/내국인(집계구) 성별연령대별.csv', encoding='cp949')
data2=pd.read_csv('/content/내국인(집계구) 유입지별.csv', encoding='cp949')
data3=pd.read_csv('/content/신한카드 내국인 63업종 코드.csv', encoding='cp949')
data4=pd.read_csv('/content/내국인(블록) 일자별시간대별.csv',encoding='cp949')

## data1 전처리

In [260]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   가맹점집계구코드(TOT_REG_CD)   500 non-null    int64  
 1   내국인업종코드(SB_UPJONG_CD)  500 non-null    object 
 2   기준년월(TS_YM)            500 non-null    int64  
 3   일별(TS_YMD)             500 non-null    int64  
 4   개인법인구분(PSN_CPR)        500 non-null    object 
 5   성별(SEX_CCD)            473 non-null    object 
 6   연령대별(AGE_GB)           468 non-null    object 
 7   카드이용금액계(AMT_CORR)      500 non-null    float64
 8   카드이용건수(USECT_CORR)     500 non-null    float64
dtypes: float64(2), int64(3), object(4)
memory usage: 35.3+ KB


In [261]:
# data1의 결측치 확인하기
data1.isnull().sum()

가맹점집계구코드(TOT_REG_CD)      0
내국인업종코드(SB_UPJONG_CD)     0
기준년월(TS_YM)               0
일별(TS_YMD)                0
개인법인구분(PSN_CPR)           0
성별(SEX_CCD)              27
연령대별(AGE_GB)             32
카드이용금액계(AMT_CORR)         0
카드이용건수(USECT_CORR)        0
dtype: int64

In [262]:
# 결측치 0으로 대체
data1= data1.fillna(0)

# 성별과 연령대 별 나이 0인 값은 제외
data1= data1.loc[data1['성별(SEX_CCD)'] != 0]
data1= data1.loc[data1['연령대별(AGE_GB)'] != 0]

# 연령대 별 나이 '대' 제외하기
data1['연령대별(AGE_GB)']= data1['연령대별(AGE_GB)'].str.replace('대','')

# 카드 이용금액계, 카드 이용건수 일의 자리에서 반올림하기
data1['카드이용건수(USECT_CORR)']= round(data1['카드이용건수(USECT_CORR)'], 0)
data1['카드이용금액계(AMT_CORR)']= round(data1['카드이용금액계(AMT_CORR)'], 0)

# 기준년월, 일별 datetime으로 단위 변경
data1['기준년월(TS_YM)'] = data1['기준년월(TS_YM)'].apply(lambda x: pd.to_datetime(x, format='%Y%m').strftime('%Y-%m'))
data1['일별(TS_YMD)']= data1['일별(TS_YMD)'].apply(lambda x: pd.to_datetime(str(x), format='%Y-%m-%d'))

# 이용건수, 카드 이용금액계 int형으로 바꾸기
data1['카드이용건수(USECT_CORR)']= data1['카드이용건수(USECT_CORR)'].astype(int)
data1['카드이용금액계(AMT_CORR)']=data1['카드이용금액계(AMT_CORR)'].astype(int)

# 빅쿼리 적재를 위한 컬럼명 변경
data1.columns=['TOT_REG_CD','SB_UPJONG_CD','TS_YM','TS_YMD','PSN_CPR','SEX_CCD','AGE_GB','AMT_CORR','USECT_CORR']

In [263]:
# # 최종 내국인 성별연령대별 데이터
# data1.to_csv('내국인(집계구) 성별연령대별_최종.csv')

## data2 전처리

In [264]:
data2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   가맹점집계구코드(TOT_REG_CD)   500 non-null    int64  
 1   내국인업종코드(SB_UPJONG_CD)  500 non-null    object 
 2   기준연월(TS_YM)            500 non-null    int64  
 3   일별(TS_YMD)             500 non-null    int64  
 4   고객주소광역시(SIDO)          500 non-null    object 
 5   고객주소시군구(SGG)           438 non-null    object 
 6   카드이용금액계(AMT_CORR)      500 non-null    float64
 7   카드이용건수(USECT_CORR)     500 non-null    float64
dtypes: float64(2), int64(3), object(3)
memory usage: 31.4+ KB


In [265]:
# data2의 결측치 확인하기
data2.isnull().sum()

가맹점집계구코드(TOT_REG_CD)      0
내국인업종코드(SB_UPJONG_CD)     0
기준연월(TS_YM)               0
일별(TS_YMD)                0
고객주소광역시(SIDO)             0
고객주소시군구(SGG)             62
카드이용금액계(AMT_CORR)         0
카드이용건수(USECT_CORR)        0
dtype: int64

In [266]:
# 결측치 '알수없음'으로 대체
data2= data2.fillna('알수없음')

# 고객 주소 시군구 '강동구서울;에서 서울 제거
data2['고객주소시군구(SGG)']= data2['고객주소시군구(SGG)'].str.replace('서울','')

# 카드 이용금액계, 카드 이용건수 일의 자리에서 반올림하기
data2['카드이용건수(USECT_CORR)']= round(data2['카드이용건수(USECT_CORR)'], 0)
data2['카드이용금액계(AMT_CORR)']= round(data2['카드이용금액계(AMT_CORR)'], 0)

# 이용건수, 카드 이용금액계 int형으로 바꾸기
data2['카드이용건수(USECT_CORR)']= data2['카드이용건수(USECT_CORR)'].astype(int)
data2['카드이용금액계(AMT_CORR)']=data2['카드이용금액계(AMT_CORR)'].astype(int)

# 기준년월, 일별 datetime으로 단위 변경
data2['기준연월(TS_YM)'] = data2['기준연월(TS_YM)'].apply(lambda x: pd.to_datetime(x, format='%Y%m').strftime('%Y-%m'))
data2['일별(TS_YMD)']= data2['일별(TS_YMD)'].apply(lambda x: pd.to_datetime(str(x), format='%Y-%m-%d'))

In [267]:
# 빅쿼리 적재를 위한 컬럼명 변경
data2.columns=['TOT_REG_CD', 'SB_UPJONG_CD', 'TS_YM', 'TS_YMD', 'SIDO', 'SGG', 'AMT_CORR', 'USECT_CORR']

In [268]:
# # 내국인(집계구) 유입지별 데이터
# data2.to_csv('내국인(집계구) 유입지별_최종.csv')

## data3 전처리

In [269]:
# data3 정보 확인
data3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63 entries, 0 to 62
Data columns (total 4 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   대분류(SB_L_UPJONG_NM)    63 non-null     object
 1   중분류(SB_M_UPJONG_NM)    63 non-null     object
 2   내국인업종분류(SB_UPJONG_NM)  63 non-null     object
 3   내국인업종코드(SB_UPJONG_CD)  63 non-null     object
dtypes: object(4)
memory usage: 2.1+ KB


In [270]:
# data3 결측치 확인하기
data3.isnull().sum()

대분류(SB_L_UPJONG_NM)      0
중분류(SB_M_UPJONG_NM)      0
내국인업종분류(SB_UPJONG_NM)    0
내국인업종코드(SB_UPJONG_CD)    0
dtype: int64

In [271]:
# 내국인업종코드(SB_UPJONG_CD) 대문자로 변경
data3['내국인업종코드(SB_UPJONG_CD)']= data3['내국인업종코드(SB_UPJONG_CD)'].str.upper()

# 빅쿼리 적재를 위한 컬럼명 변경
data3.columns=['SB_L_UPJONG_NM','SB_M_UPJONG_NM','SB_UPJONG_NM','SB_UPJONG_CD' ]

In [272]:
# # 신한카드 내국인 63업종 코드 데이터
# data3.to_csv('신한카드 내국인 63업종 코드_최종.csv')

## data4 전처리


In [273]:
# data4 정보 확인
data4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   가맹점블록코드(BLCK_CD)       500 non-null    int64  
 1   내국인업종코드(SB_UPJONG_CD)  500 non-null    object 
 2   기준년월(TS_YM)            500 non-null    int64  
 3   일별(TS_YMD)             500 non-null    int64  
 4   요일(DAW)                500 non-null    object 
 5   시간대(TM)                500 non-null    int64  
 6   카드이용금액계(AMT_CORR)      500 non-null    float64
 7   카드이용건수(USECT_CORR)     500 non-null    float64
dtypes: float64(2), int64(4), object(2)
memory usage: 31.4+ KB


In [274]:
# data4 결측치 확인하기
data4.isnull().sum()

가맹점블록코드(BLCK_CD)         0
내국인업종코드(SB_UPJONG_CD)    0
기준년월(TS_YM)              0
일별(TS_YMD)               0
요일(DAW)                  0
시간대(TM)                  0
카드이용금액계(AMT_CORR)        0
카드이용건수(USECT_CORR)       0
dtype: int64

In [275]:
# 기준년월, 일별 datetime으로 변경
data4['기준년월(TS_YM)'] = data4['기준년월(TS_YM)'].apply(lambda x: pd.to_datetime(x, format='%Y%m').strftime('%Y-%m'))
data4['일별(TS_YMD)']= data4['일별(TS_YMD)'].apply(lambda x: pd.to_datetime(str(x), format='%Y-%m-%d'))

# 카드 이용 금액계, 카드 이용 건수 일의 자리에서 반올림
data4['카드이용건수(USECT_CORR)']= round(data4['카드이용건수(USECT_CORR)'], 0)
data4['카드이용금액계(AMT_CORR)']= round(data4['카드이용금액계(AMT_CORR)'], 0)

# 이용건수, 카드 이용금액계 int형으로 바꾸기
data4['카드이용건수(USECT_CORR)']= data4['카드이용건수(USECT_CORR)'].astype(int)
data4['카드이용금액계(AMT_CORR)']=data4['카드이용금액계(AMT_CORR)'].astype(int)

# 빅쿼리 적재를 위한 컬럼명 변경
data4.columns=['BLCK_CD','SB_UPJONG_CD','TS_YM','TS_YMD','DAW','TM','AMT_CORR','USECT_CORR']

In [276]:
# # 내국인(블록) 일자별시간대별 데이터
# data4.to_csv('내국인(블록) 일자별시간대별_최종.csv')